In [0]:
import dlt
from pyspark.sql.functions import col,expr,lit
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, StructField, IntegerType
from pyspark.sql.session import SparkSession

In [0]:
schema = StructType(
    [
        StructField("pkey", IntegerType(), False),
        StructField("Description", StringType(), False),
        StructField("Name", StringType(), False),
        StructField("value", StringType(), False)

    ]
)

cloud_file_options = {
    "cloudFiles.format":"csv"
}

In [0]:
@dlt.table(
    name="bronze_load"
)
def bronze_load():
    df=spark.readStream.format("cloudFiles").options(**cloud_file_options).schema(schema).load("/Volumes/awsdbx_w1_2358208440317044/default/cdc_v3/")
    df=df.withColumn("file_processed_date",F.date_format(F.current_timestamp(),"yyyy-MM-dd HH:mm:ss"))
    return df


In [0]:
dlt.create_streaming_table(
    name="silver_load"
)

In [0]:
dlt.apply_changes(
    target="silver_load",
    source="bronze_load",
    keys=["pkey"],
    stored_as_scd_type="1",
    sequence_by="file_processed_date"
)

In [0]:
#spark.read.table("awsdbx_w1_2358208440317044.default.bronze_load").display()

In [0]:
#spark.read.table("`awsdbx_w1_2358208440317044`.`default`.silver_load").display()